In [1]:
import os
import time
import glob
import pathlib
import pandas as pd

In [2]:
### Create a list of all csv files in our data folder

csv1 = glob.glob(os.getcwd() + '/data' + '/**/*.csv', recursive=True)

In [3]:
### Create a dictionary containing local codes and corresponding csv files

stat_dict = {}
for file in csv1:
    stat_dict[int(file.split('_')[-3])] = file

In [4]:
### Create dataframe from all csv files, add local_codes, then drop duplicates so that only unique stations remain

df = pd.concat([pd.read_csv(f, nrows=1) for f in list(stat_dict.values())])
local_code_list = []
for i in range(len(list(stat_dict.values()))):
    local_code_list.append(list(stat_dict.values())[i].split('_')[-3])
df['local_code'] = local_code_list
df.drop_duplicates(['AirQualityStation'], keep='first', inplace=True)

In [5]:
### Steps to prepare merged_metadata from PanEuropean and AirBase_v8 files
### Metadata information is combined, leaving only one row per unique station

# df_meta = pd.read_csv('PanEuropean_metadata.csv', sep="\t")
# df_stat = pd.read_csv('AirBase_v8_stations.csv', sep="\t")
# drop_list2 = ['Countrycode','Timezone','Namespace','AirQualityNetwork','AirQualityStationNatCode','SamplingPoint','SamplingProces','Sample','AirPollutantCode','ObservationDateBegin','ObservationDateEnd','Projection','Altitude','MeasurementType','EquivalenceDemonstrated','MeasurementEquipment','InletHeight','BuildingDistance','KerbDistance']
# drop_list3 = ['station_local_code','type_of_station','station_ozone_classification','station_type_of_area','station_subcat_rural_back','street_type','station_longitude_deg','station_latitude_deg','station_altitude','lau_level1_code','lau_level2_code','lau_level2_name','EMEP_station']
# df_meta.drop(columns=drop_list2, inplace=True)
# df_stat.drop(columns=drop_list3, inplace=True)
# df_meta.drop_duplicates(subset='AirQualityStation', keep='first', inplace=True)
# df_stat.drop_duplicates(subset='station_european_code', keep='first', inplace=True)
# df_stat.rename(columns = {'station_european_code':'AirQualityStationEoICode',
#                           'country_iso_code':'Countrycode',
#                           'country_name':'CountryName',
#                           'station_name':'StationName',
#                           'station_start_date':'StationStartDate',
#                           'station_end_date':'StationEndDate',
#                           'station_city':'StationCity'}, inplace = True)
# meta = df_meta.merge(df_stat, on='AirQualityStationEoICode', how='left')
# meta.to_csv( "merged_metadata.csv", index=False, encoding='utf-8-sig')

In [6]:
### Load previously created merged_metadata (PanEuropean merged with AirBase_v8 containing only unique stations)

metadata = pd.read_csv("merged_metadata.csv")
metadata = metadata.reset_index().rename(columns={'index': 'LocalCode'})

In [7]:
### Add local_code to merged_metadata then prepare it for subsequent merge with PanEuropean

new_meta = metadata.merge(df, on='AirQualityStation', how='left')
new_meta['LocalCode'] = new_meta['local_code']
new_meta.drop(columns=['local_code'], inplace=True)
new_meta = new_meta.loc[:, ['LocalCode', 'AirQualityStation', 'CountryName', 'StationName', \
                            'StationStartDate', 'StationEndDate', 'StationCity']]

In [8]:
### Merge PanEuropean with previously created metadata to create a final full metadata file

PanEuropean_metadata = pd.read_csv('PanEuropean_metadata.csv', sep="\t")
new_meta_full = PanEuropean_metadata.merge(new_meta, on='AirQualityStation', how='left')

In [9]:
### Export final metadata file

new_meta_full.to_csv("final_metadata.csv", index=False, encoding='utf-8-sig')